# 📊 AB Test Template: From Raw Data to Power BI Dashboard

This notebook demonstrates the complete workflow from raw data collection to statistical analysis and Power BI visualization.

## 🎯 What This Template Covers

1. **Raw Data Generation** - Realistic AB test data simulation
2. **Data Preprocessing** - Cleaning and preparation
3. **Statistical Analysis** - Comprehensive AB test analysis  
4. **Power BI Export** - Structured data for dashboards
5. **Visualization Guidelines** - Power BI dashboard templates

## 📋 Sample Scenario

**Company:** E-commerce website  
**Test:** New checkout button design vs. current design  
**Metric:** Conversion rate (purchases/visitors)  
**Duration:** 14 days  
**Traffic Split:** 50/50  

Let's build this step by step!

## 📚 Step 1: Import Required Libraries

In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind, chi2_contingency, norm, beta
import math
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configure visualization styles
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("📊 Ready for AB Test Analysis Template")

✅ All libraries imported successfully!
📊 Ready for AB Test Analysis Template


## 🎲 Step 2: Generate Realistic Sample Data

This simulates raw data collection from a real AB test scenario.

In [2]:
# Generate realistic AB test data
def generate_realistic_ab_test_data():
    """
    Generate realistic AB test data for an e-commerce checkout button test
    """
    np.random.seed(42)  # For reproducibility
    
    # Test parameters
    test_duration_days = 14
    daily_visitors = 800
    total_visitors = test_duration_days * daily_visitors
    
    # Control group parameters (current design)
    control_conversion_rate = 0.12  # 12% baseline conversion
    control_avg_order_value = 85
    
    # Treatment group parameters (new design) - slightly better
    treatment_conversion_rate = 0.145  # 14.5% conversion (+20.8% relative improvement)
    treatment_avg_order_value = 88  # Slightly higher AOV
    
    # Generate user data
    user_data = []
    
    start_date = datetime(2025, 10, 15)
    
    for day in range(test_duration_days):
        current_date = start_date + timedelta(days=day)
        
        # Simulate daily traffic variation (weekends have different patterns)
        if current_date.weekday() >= 5:  # Weekend
            daily_traffic = int(daily_visitors * 0.8)  # 20% less traffic on weekends
        else:
            daily_traffic = daily_visitors
            
        # Add some randomness to daily traffic
        daily_traffic = int(daily_traffic * np.random.uniform(0.85, 1.15))
        
        for visitor_id in range(daily_traffic):
            # Random assignment to groups (50/50 split)
            group = np.random.choice(['Control', 'Treatment'])
            
            # Generate timestamp within the day
            hour = np.random.randint(8, 23)  # Business hours 8 AM - 11 PM
            minute = np.random.randint(0, 60)
            timestamp = current_date.replace(hour=hour, minute=minute)
            
            # Determine conversion based on group
            if group == 'Control':
                converted = np.random.binomial(1, control_conversion_rate)
                base_order_value = control_avg_order_value
            else:
                converted = np.random.binomial(1, treatment_conversion_rate)
                base_order_value = treatment_avg_order_value
            
            # Generate revenue if converted
            if converted:
                # Revenue follows a gamma distribution (realistic for e-commerce)
                revenue = np.random.gamma(2, base_order_value/2)
                revenue = max(10, revenue)  # Minimum order value
            else:
                revenue = 0
            
            # Generate user demographics and behavior
            user_data.append({
                'user_id': f"user_{day:02d}_{visitor_id:04d}",
                'timestamp': timestamp,
                'date': current_date.date(),
                'day_of_week': current_date.strftime('%A'),
                'group': group,
                'converted': converted,
                'revenue': revenue,
                'device': np.random.choice(['Desktop', 'Mobile', 'Tablet'], p=[0.6, 0.35, 0.05]),
                'traffic_source': np.random.choice(['Organic', 'Paid Search', 'Social', 'Direct', 'Email'], 
                                                 p=[0.35, 0.25, 0.15, 0.15, 0.10]),
                'new_customer': np.random.choice([True, False], p=[0.7, 0.3]),
                'page_views': np.random.poisson(4) + 1,  # Number of pages viewed
                'time_on_site': np.random.gamma(2, 60)  # Time in seconds
            })
    
    return pd.DataFrame(user_data)

# Generate the data
print("🎲 Generating realistic AB test data...")
df = generate_realistic_ab_test_data()

print(f"✅ Generated {len(df):,} user records over {df['date'].nunique()} days")
print(f"📊 Data shape: {df.shape}")
print(f"🏷️ Groups: {df['group'].value_counts().to_dict()}")

# Display sample data
print("\n📋 Sample Data:")
print(df.head(10))

print(f"\n📈 Quick Overview:")
print(f"Total Conversions: {df['converted'].sum():,}")
print(f"Total Revenue: ${df['revenue'].sum():,.2f}")
print(f"Date Range: {df['date'].min()} to {df['date'].max()}")

🎲 Generating realistic AB test data...
✅ Generated 10,131 user records over 14 days
📊 Data shape: (10131, 12)
🏷️ Groups: {np.str_('Treatment'): 5068, np.str_('Control'): 5063}

📋 Sample Data:
        user_id           timestamp        date day_of_week      group  \
0  user_00_0000 2025-10-15 22:42:00  2025-10-15   Wednesday    Control   
1  user_00_0001 2025-10-15 19:20:00  2025-10-15   Wednesday  Treatment   
2  user_00_0002 2025-10-15 21:46:00  2025-10-15   Wednesday    Control   
3  user_00_0003 2025-10-15 12:01:00  2025-10-15   Wednesday  Treatment   
4  user_00_0004 2025-10-15 21:05:00  2025-10-15   Wednesday  Treatment   
5  user_00_0005 2025-10-15 19:59:00  2025-10-15   Wednesday    Control   
6  user_00_0006 2025-10-15 15:34:00  2025-10-15   Wednesday    Control   
7  user_00_0007 2025-10-15 18:34:00  2025-10-15   Wednesday  Treatment   
8  user_00_0008 2025-10-15 14:48:00  2025-10-15   Wednesday    Control   
9  user_00_0009 2025-10-15 21:56:00  2025-10-15   Wednesday    Contr

## 🧹 Step 3: Data Preprocessing and Exploration

In [3]:
# Data preprocessing and exploration
print("🔍 DATA EXPLORATION AND PREPROCESSING")
print("=" * 50)

# 1. Basic data info
print("\n1. DATASET OVERVIEW")
print("-" * 30)
print(f"📊 Dataset shape: {df.shape}")
print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")
print(f"🎯 Total users: {len(df):,}")
print(f"💰 Total revenue: ${df['revenue'].sum():,.2f}")

# 2. Data types and missing values
print(f"\n2. DATA QUALITY CHECK")
print("-" * 30)
print("Data types:")
print(df.dtypes)
print(f"\nMissing values:")
print(df.isnull().sum())

# 3. Group balance
print(f"\n3. GROUP DISTRIBUTION")
print("-" * 30)
group_stats = df.groupby('group').agg({
    'user_id': 'count',
    'converted': ['sum', 'mean'],
    'revenue': ['sum', 'mean']
}).round(4)

group_stats.columns = ['Users', 'Conversions', 'Conv_Rate', 'Total_Revenue', 'Avg_Revenue_Per_User']
print(group_stats)

# 4. Daily trends
print(f"\n4. DAILY TRENDS")
print("-" * 30)
daily_stats = df.groupby(['date', 'group']).agg({
    'user_id': 'count',
    'converted': ['sum', 'mean']
}).round(4)

daily_stats.columns = ['Users', 'Conversions', 'Conv_Rate']
print("Sample daily data:")
print(daily_stats.head(8))

# 5. Device and traffic source breakdown
print(f"\n5. SEGMENT ANALYSIS")
print("-" * 30)
print("By Device:")
device_analysis = df.groupby(['device', 'group'])['converted'].mean().unstack().round(4)
print(device_analysis)

print("\nBy Traffic Source:")
traffic_analysis = df.groupby(['traffic_source', 'group'])['converted'].mean().unstack().round(4)
print(traffic_analysis)

# 6. Weekend vs Weekday patterns
print(f"\n6. TEMPORAL PATTERNS")
print("-" * 30)
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])
temporal_analysis = df.groupby(['is_weekend', 'group'])['converted'].mean().unstack().round(4)
temporal_analysis.index = ['Weekday', 'Weekend']
print(temporal_analysis)

print("\n✅ Data exploration complete!")
print("📊 Data is clean and ready for statistical analysis")

🔍 DATA EXPLORATION AND PREPROCESSING

1. DATASET OVERVIEW
------------------------------
📊 Dataset shape: (10131, 12)
📅 Date range: 2025-10-15 to 2025-10-28
🎯 Total users: 10,131
💰 Total revenue: $121,638.22

2. DATA QUALITY CHECK
------------------------------
Data types:
user_id                   object
timestamp         datetime64[ns]
date                      object
day_of_week               object
group                     object
converted                  int64
revenue                  float64
device                    object
traffic_source            object
new_customer                bool
page_views                 int64
time_on_site             float64
dtype: object

Missing values:
user_id           0
timestamp         0
date              0
day_of_week       0
group             0
converted         0
revenue           0
device            0
traffic_source    0
new_customer      0
page_views        0
time_on_site      0
dtype: int64

3. GROUP DISTRIBUTION
-----------------------

## 📈 Step 4: Statistical Analysis (Your Proven Methods)

In [4]:
# Statistical Analysis - Using your proven methods
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Extract the key numbers for analysis
control_data = df[df['group'] == 'Control']
treatment_data = df[df['group'] == 'Treatment']

control_users = len(control_data)
control_conversions = control_data['converted'].sum()
treatment_users = len(treatment_data)
treatment_conversions = treatment_data['converted'].sum()

print("🧪 COMPREHENSIVE AB TEST STATISTICAL ANALYSIS")
print("=" * 60)

# 1. BASIC STATISTICS
print("\n1. BASIC STATISTICS")
print("-" * 30)
control_rate = control_conversions / control_users
treatment_rate = treatment_conversions / treatment_users
difference = treatment_rate - control_rate
relative_change = (difference / control_rate) * 100

print(f"Control: {control_conversions}/{control_users} = {control_rate:.4f} ({control_rate*100:.2f}%)")
print(f"Treatment: {treatment_conversions}/{treatment_users} = {treatment_rate:.4f} ({treatment_rate*100:.2f}%)")
print(f"Absolute difference: {difference:.4f} ({difference*100:.2f} percentage points)")
print(f"Relative change: {relative_change:.1f}%")

# 2. STATISTICAL SIGNIFICANCE TESTING
print("\n2. STATISTICAL SIGNIFICANCE")
print("-" * 30)

# Two-proportion z-test
successes = [control_conversions, treatment_conversions]
totals = [control_users, treatment_users]
z_stat, p_value = proportions_ztest(successes, totals)

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print(f"Significant at α=0.05: {'YES' if p_value < 0.05 else 'NO'}")

# 3. CONFIDENCE INTERVALS
print("\n3. CONFIDENCE INTERVALS")
print("-" * 30)

# Individual CIs
control_ci = proportion_confint(control_conversions, control_users, alpha=0.05, method='wilson')
treatment_ci = proportion_confint(treatment_conversions, treatment_users, alpha=0.05, method='wilson')

print(f"Control 95% CI: [{control_ci[0]:.4f}, {control_ci[1]:.4f}] = [{control_ci[0]*100:.2f}%, {control_ci[1]*100:.2f}%]")
print(f"Treatment 95% CI: [{treatment_ci[0]:.4f}, {treatment_ci[1]:.4f}] = [{treatment_ci[0]*100:.2f}%, {treatment_ci[1]*100:.2f}%]")

# Difference CI (approximation)
se_diff = np.sqrt(control_rate*(1-control_rate)/control_users + treatment_rate*(1-treatment_rate)/treatment_users)
diff_ci_lower = difference - 1.96 * se_diff
diff_ci_upper = difference + 1.96 * se_diff
print(f"Difference 95% CI: [{diff_ci_lower:.4f}, {diff_ci_upper:.4f}] = [{diff_ci_lower*100:.2f}pp, {diff_ci_upper*100:.2f}pp]")

# 4. EFFECT SIZE (Cohen's h for proportions)
print("\n4. EFFECT SIZE")
print("-" * 30)
cohens_h = 2 * (np.arcsin(np.sqrt(treatment_rate)) - np.arcsin(np.sqrt(control_rate)))
print(f"Cohen's h: {cohens_h:.4f}")
if abs(cohens_h) < 0.2:
    effect_magnitude = "Small"
elif abs(cohens_h) < 0.5:
    effect_magnitude = "Medium"
else:
    effect_magnitude = "Large"
print(f"Effect magnitude: {effect_magnitude}")

# 5. POWER ANALYSIS
print("\n5. POWER ANALYSIS")
print("-" * 30)

def calculate_power(n1, n2, p1, p2, alpha=0.05):
    """Calculate statistical power for two-proportion test"""
    pooled_p = (n1*p1 + n2*p2) / (n1 + n2)
    se_null = np.sqrt(pooled_p * (1 - pooled_p) * (1/n1 + 1/n2))
    se_alt = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = (abs(p1 - p2) - z_alpha * se_null) / se_alt
    power = stats.norm.cdf(z_beta)
    
    return max(0, min(1, power))

current_power = calculate_power(control_users, treatment_users, control_rate, treatment_rate)
print(f"Current statistical power: {current_power:.3f} ({current_power*100:.1f}%)")

# 6. BAYESIAN ANALYSIS
print("\n6. BAYESIAN ANALYSIS")
print("-" * 30)

# Beta-binomial model with uniform priors
alpha_control = 1 + control_conversions
beta_control = 1 + control_users - control_conversions
alpha_treatment = 1 + treatment_conversions
beta_treatment = 1 + treatment_users - treatment_conversions

# Probability that treatment is better than control
n_samples = 100000
control_samples = np.random.beta(alpha_control, beta_control, n_samples)
treatment_samples = np.random.beta(alpha_treatment, beta_treatment, n_samples)

prob_treatment_better = np.mean(treatment_samples > control_samples)
prob_treatment_much_better = np.mean(treatment_samples > 1.1 * control_samples)  # >10% better

print(f"Probability treatment is better than control: {prob_treatment_better:.4f} ({prob_treatment_better*100:.1f}%)")
print(f"Probability treatment is >10% better: {prob_treatment_much_better:.4f} ({prob_treatment_much_better*100:.1f}%)")

# 7. SAMPLE SIZE PLANNING
print("\n7. SAMPLE SIZE PLANNING FOR FUTURE TESTS")
print("-" * 30)

def sample_size_calculator(p1, p2, alpha=0.05, power=0.8):
    """Calculate required sample size per group"""
    p_pooled = (p1 + p2) / 2
    effect_size = abs(p1 - p2)
    
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    n = (z_alpha + z_beta)**2 * p_pooled * (1 - p_pooled) / (effect_size**2)
    return math.ceil(n)

# Store results for Power BI export
analysis_results = {
    'control_users': control_users,
    'control_conversions': control_conversions,
    'treatment_users': treatment_users, 
    'treatment_conversions': treatment_conversions,
    'control_rate': control_rate,
    'treatment_rate': treatment_rate,
    'difference': difference,
    'relative_change': relative_change,
    'z_stat': z_stat,
    'p_value': p_value,
    'cohens_h': cohens_h,
    'current_power': current_power,
    'prob_treatment_better': prob_treatment_better,
    'control_ci': control_ci,
    'treatment_ci': treatment_ci
}

print("✅ Statistical analysis complete!")
print(f"📊 Key Result: Treatment shows {relative_change:.1f}% {'improvement' if relative_change > 0 else 'decline'}")
print(f"🎯 Significance: {'SIGNIFICANT' if p_value < 0.05 else 'NOT SIGNIFICANT'} (p={p_value:.4f})")

🧪 COMPREHENSIVE AB TEST STATISTICAL ANALYSIS

1. BASIC STATISTICS
------------------------------
Control: 596/5063 = 0.1177 (11.77%)
Treatment: 777/5068 = 0.1533 (15.33%)
Absolute difference: 0.0356 (3.56 percentage points)
Relative change: 30.2%

2. STATISTICAL SIGNIFICANCE
------------------------------
Z-statistic: -5.2341
P-value: 0.000000
Significant at α=0.05: YES

3. CONFIDENCE INTERVALS
------------------------------
Control 95% CI: [0.1091, 0.1269] = [10.91%, 12.69%]
Treatment 95% CI: [0.1437, 0.1635] = [14.37%, 16.35%]
Difference 95% CI: [0.0223, 0.0489] = [2.23pp, 4.89pp]

4. EFFECT SIZE
------------------------------
Cohen's h: 0.1042
Effect magnitude: Small

5. POWER ANALYSIS
------------------------------
Current statistical power: 0.999 (99.9%)

6. BAYESIAN ANALYSIS
------------------------------
Probability treatment is better than control: 1.0000 (100.0%)
Probability treatment is >10% better: 0.9996 (100.0%)

7. SAMPLE SIZE PLANNING FOR FUTURE TESTS
-------------------

## 📊 Step 5: Export Data for Power BI

In [5]:
# Create Power BI datasets
import os

# Create output directory
output_dir = 'powerbi_data'
os.makedirs(output_dir, exist_ok=True)

print("📊 CREATING POWER BI DATASETS")
print("=" * 40)

# 1. SUMMARY STATISTICS TABLE
summary_stats = pd.DataFrame({
    'Metric': [
        'Control Users',
        'Treatment Users',
        'Control Conversions', 
        'Treatment Conversions',
        'Control Conversion Rate',
        'Treatment Conversion Rate',
        'Absolute Difference',
        'Relative Improvement %',
        'Z Statistic',
        'P Value',
        'Statistical Significant',
        'Effect Size (Cohen h)',
        'Statistical Power',
        'Prob Treatment Better',
        'Test Duration (Days)',
        'Total Revenue Control',
        'Total Revenue Treatment',
        'Revenue Per User Control',
        'Revenue Per User Treatment'
    ],
    'Value': [
        control_users,
        treatment_users,
        control_conversions,
        treatment_conversions,
        f"{control_rate:.4f}",
        f"{treatment_rate:.4f}",
        f"{difference:.4f}", 
        f"{relative_change:.2f}",
        f"{z_stat:.4f}",
        f"{p_value:.6f}",
        'Yes' if p_value < 0.05 else 'No',
        f"{cohens_h:.4f}",
        f"{current_power:.3f}",
        f"{prob_treatment_better:.3f}",
        df['date'].nunique(),
        f"{control_data['revenue'].sum():.2f}",
        f"{treatment_data['revenue'].sum():.2f}",
        f"{control_data['revenue'].sum()/control_users:.2f}",
        f"{treatment_data['revenue'].sum()/treatment_users:.2f}"
    ],
    'Category': [
        'Sample Size', 'Sample Size', 'Conversions', 'Conversions',
        'Rate', 'Rate', 'Difference', 'Performance',
        'Statistical Test', 'Statistical Test', 'Result', 'Effect',
        'Power', 'Bayesian', 'Duration',
        'Revenue', 'Revenue', 'Revenue', 'Revenue'
    ]
})

# 2. DAILY PERFORMANCE DATA
daily_data = df.groupby(['date', 'group']).agg({
    'user_id': 'count',
    'converted': ['sum', 'mean'],
    'revenue': 'sum'
}).reset_index()

daily_data.columns = ['Date', 'Group', 'Users', 'Conversions', 'Conversion_Rate', 'Revenue']
daily_data['Day_Number'] = daily_data['Date'].rank(method='dense').astype(int)
daily_data['Weekday'] = pd.to_datetime(daily_data['Date']).dt.day_name()

# 3. CONFIDENCE INTERVALS DATA
ci_data = pd.DataFrame({
    'Group': ['Control', 'Treatment'],
    'Conversion_Rate': [control_rate, treatment_rate],
    'Users': [control_users, treatment_users],
    'Conversions': [control_conversions, treatment_conversions],
    'CI_Lower': [control_ci[0], treatment_ci[0]],
    'CI_Upper': [control_ci[1], treatment_ci[1]],
    'Margin_of_Error': [control_ci[1] - control_rate, treatment_ci[1] - treatment_rate]
})

# 4. SEGMENT ANALYSIS DATA
segment_data = []

# Device analysis
for device in df['device'].unique():
    for group in ['Control', 'Treatment']:
        subset = df[(df['device'] == device) & (df['group'] == group)]
        if len(subset) > 0:
            segment_data.append({
                'Segment_Type': 'Device',
                'Segment_Value': device,
                'Group': group,
                'Users': len(subset),
                'Conversions': subset['converted'].sum(),
                'Conversion_Rate': subset['converted'].mean(),
                'Revenue': subset['revenue'].sum()
            })

# Traffic source analysis  
for source in df['traffic_source'].unique():
    for group in ['Control', 'Treatment']:
        subset = df[(df['traffic_source'] == source) & (df['group'] == group)]
        if len(subset) > 0:
            segment_data.append({
                'Segment_Type': 'Traffic_Source',
                'Segment_Value': source,
                'Group': group,
                'Users': len(subset),
                'Conversions': subset['converted'].sum(),
                'Conversion_Rate': subset['converted'].mean(),
                'Revenue': subset['revenue'].sum()
            })

segment_df = pd.DataFrame(segment_data)

# 5. RAW USER DATA (SAMPLE FOR POWER BI)
# Include a sample of raw data for drill-down analysis
raw_sample = df.sample(n=min(5000, len(df)), random_state=42).copy()
raw_sample['Revenue_Per_User'] = raw_sample['revenue']
raw_sample['Converted_Text'] = raw_sample['converted'].map({0: 'No', 1: 'Yes'})

# Export all datasets
summary_stats.to_csv(f'{output_dir}/ab_test_summary.csv', index=False)
daily_data.to_csv(f'{output_dir}/daily_performance.csv', index=False)
ci_data.to_csv(f'{output_dir}/confidence_intervals.csv', index=False) 
segment_df.to_csv(f'{output_dir}/segment_analysis.csv', index=False)
raw_sample.to_csv(f'{output_dir}/raw_user_sample.csv', index=False)

print("✅ Power BI datasets created successfully!")
print(f"\n📂 Files created in '{output_dir}' folder:")
print("  1. ab_test_summary.csv - Key metrics and statistical results")
print("  2. daily_performance.csv - Daily trends and performance")
print("  3. confidence_intervals.csv - Statistical confidence data")
print("  4. segment_analysis.csv - Device and traffic source breakdown")
print("  5. raw_user_sample.csv - Sample raw data for drill-down")

print(f"\n📊 Summary Dataset Preview:")
print(summary_stats.head(10))

print(f"\n📈 Daily Performance Sample:")
print(daily_data.head(6))

📊 CREATING POWER BI DATASETS
✅ Power BI datasets created successfully!

📂 Files created in 'powerbi_data' folder:
  1. ab_test_summary.csv - Key metrics and statistical results
  2. daily_performance.csv - Daily trends and performance
  3. confidence_intervals.csv - Statistical confidence data
  4. segment_analysis.csv - Device and traffic source breakdown
  5. raw_user_sample.csv - Sample raw data for drill-down

📊 Summary Dataset Preview:
                      Metric     Value          Category
0              Control Users      5063       Sample Size
1            Treatment Users      5068       Sample Size
2        Control Conversions       596       Conversions
3      Treatment Conversions       777       Conversions
4    Control Conversion Rate    0.1177              Rate
5  Treatment Conversion Rate    0.1533              Rate
6        Absolute Difference    0.0356        Difference
7     Relative Improvement %     30.24       Performance
8                Z Statistic   -5.2341  St

In [ ]:
# 📊 AB Test Template: From Raw Data to Power BI Dashboard

This notebook demonstrates the complete workflow from raw data collection to statistical analysis and Power BI visualization.

## 🎯 What This Template Covers

1. **Raw Data Generation** - Realistic AB test data simulation
2. **Data Preprocessing** - Cleaning and preparation
3. **Statistical Analysis** - Comprehensive AB test analysis  
4. **Power BI Export** - Structured data for dashboards
5. **Visualization Guidelines** - Power BI dashboard templates

## 📋 Sample Scenario

**Company:** E-commerce website  
**Test:** New checkout button design vs. current design  
**Metric:** Conversion rate (purchases/visitors)  
**Duration:** 14 days  
**Traffic Split:** 50/50  

Let's build this step by step!